<a href="https://colab.research.google.com/github/Sathya1926/Object-Detection-Using-ESP32-cam/blob/main/animal_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Create a directory to unzip into
os.makedirs("/content/dataset", exist_ok=True)

# Unzip the animal and birds datasets
!unzip "/content/drive/MyDrive/ML_DataSet /animals.zip" -d "/content/dataset/animals/"
!unzip "/content/drive/MyDrive/ML_DataSet /birds.zip" -d "/content/dataset/birds/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/animals/__MACOSX/AnimalImages/squirrel/._1d6eceed78.jpg  
  inflating: /content/dataset/animals/AnimalImages/squirrel/35e8e447e8.jpg  
  inflating: /content/dataset/animals/__MACOSX/AnimalImages/squirrel/._35e8e447e8.jpg  
  inflating: /content/dataset/animals/AnimalImages/squirrel/6ec5bd39dc.jpg  
  inflating: /content/dataset/animals/__MACOSX/AnimalImages/squirrel/._6ec5bd39dc.jpg  
  inflating: /content/dataset/animals/AnimalImages/squirrel/5b8729a6de.jpg  
  inflating: /content/dataset/animals/__MACOSX/AnimalImages/squirrel/._5b8729a6de.jpg  
  inflating: /content/dataset/animals/AnimalImages/squirrel/4ca3440732.jpg  
  inflating: /content/dataset/animals/__MACOSX/AnimalImages/squirrel/._4ca3440732.jpg  
  inflating: /content/dataset/animals/AnimalImages/squirrel/61c54deffe.jpg  
  inflating: /content/dataset/animals/__MACOSX/AnimalImages/squirrel/._61c54deffe.jpg  
  inflating: /content/dataset/animal

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set image size and batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Create training and validation generators for animals
animal_datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

animal_train = animal_datagen.flow_from_directory(
    "/content/dataset/animals",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset="training",
    class_mode="categorical"
)

animal_val = animal_datagen.flow_from_directory(
    "/content/dataset/animals",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset="validation",
    class_mode="categorical"
)

# Create training and validation generators for birds
bird_datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

bird_train = bird_datagen.flow_from_directory(
    "/content/dataset/birds",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset="training",
    class_mode="categorical"
)

bird_val = bird_datagen.flow_from_directory(
    "/content/dataset/birds",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset="validation",
    class_mode="categorical"
)

Found 1920 images belonging to 2 classes.
Found 480 images belonging to 2 classes.
Found 3614 images belonging to 2 classes.
Found 902 images belonging to 2 classes.


IInd code Run the following cell to import libraries and create ImageDataGenerators for animals and birds

In [ ]:
import os
from PIL import Image

def remove_corrupt_images(directory):
    num_removed = 0
    for folder_name in os.listdir(directory):
        folder_path = os.path.join(directory, folder_name)
        if not os.path.isdir(folder_path):
            continue
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            # ✅ Skip if it's a directory
            if os.path.isdir(file_path):
                continue
            try:
                with Image.open(file_path) as img:
                    img.verify()
            except Exception as e:
                print(f"Removing corrupt image: {file_path} ({e})")
                os.remove(file_path)
                num_removed += 1
    print(f"\n✅ Done. Removed {num_removed} corrupt or non-image files.")

# Clean the animals dataset
remove_corrupt_images("/content/dataset/animals")


✅ Done. Removed 0 corrupt or non-image files.


In [ ]:
import tensorflow as tf
import os
from PIL import Image
from pathlib import Path

# Re-check dataset path
animal_data_path = "/content/dataset/animals"

# Step 1: Remove all corrupted images
def verify_and_clean_images(root_path):
    count = 0
    for folder in Path(root_path).iterdir():
        if folder.is_dir():
            for img_path in folder.iterdir():
                if img_path.is_file() and not img_path.name.startswith('.'):
                    try:
                        img = Image.open(img_path)
                        img.verify()
                    except Exception as e:
                        print(f"❌ Removing: {img_path} ({e})")
                        img_path.unlink()
                        count += 1
    print(f"✅ Removed {count} bad image files.")

verify_and_clean_images(animal_data_path)

✅ Removed 0 bad image files.


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

animal_train = tf.keras.preprocessing.image_dataset_from_directory(
    animal_data_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

animal_val = tf.keras.preprocessing.image_dataset_from_directory(
    animal_data_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 2400 files belonging to 2 classes.
Using 1920 files for training.
Found 2400 files belonging to 2 classes.
Using 480 files for validation.


In [ ]:
# Load the MobileNetV2 base model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

# Build model
animal_model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(animal_train.cardinality().numpy(), activation='softmax')
])

# Compile model
animal_model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

# ✅ Train the model
animal_model.fit(animal_train,
                 validation_data=animal_val,
                 epochs=5)

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_multi_step_on_iterator_20830]

In [ ]:
from PIL import Image
import os
from pathlib import Path

def clean_dataset_images(directory):
    removed_count = 0
    for root, dirs, files in os.walk(directory):
        for name in files:
            file_path = os.path.join(root, name)

            # Skip hidden or system files
            if name.startswith('.') or '__MACOSX' in file_path:
                print(f"🗑 Removing system file: {file_path}")
                os.remove(file_path)
                removed_count += 1
                continue

            # Try opening the file as an image
            try:
                with Image.open(file_path) as img:
                    img.verify()  # Verify if it's a valid image
            except Exception as e:
                print(f"❌ Removing invalid image: {file_path} ({e})")
                os.remove(file_path)
                removed_count += 1

    print(f"\n✅ Cleaned dataset. Removed {removed_count} corrupt or non-image files.")

# 👇 Pass your dataset path here
clean_dataset_images("/content/dataset/animals")

🗑 Removing system file: /content/dataset/animals/__MACOSX/AnimalImages/._turtle
🗑 Removing system file: /content/dataset/animals/__MACOSX/AnimalImages/._starfish
🗑 Removing system file: /content/dataset/animals/__MACOSX/AnimalImages/._zebra
🗑 Removing system file: /content/dataset/animals/__MACOSX/AnimalImages/._snake
🗑 Removing system file: /content/dataset/animals/__MACOSX/AnimalImages/._wombat
🗑 Removing system file: /content/dataset/animals/__MACOSX/AnimalImages/._swan
🗑 Removing system file: /content/dataset/animals/__MACOSX/AnimalImages/._squid
🗑 Removing system file: /content/dataset/animals/__MACOSX/AnimalImages/._butterfly
🗑 Removing system file: /content/dataset/animals/__MACOSX/AnimalImages/._wolf
🗑 Removing system file: /content/dataset/animals/__MACOSX/AnimalImages/._sparrow
🗑 Removing system file: /content/dataset/animals/__MACOSX/AnimalImages/._boar
🗑 Removing system file: /content/dataset/animals/__MACOSX/AnimalImages/._squirrel
🗑 Removing system file: /content/dataset/

In [ ]:
animal_train = tf.keras.preprocessing.image_dataset_from_directory(
    animal_data_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

animal_val = tf.keras.preprocessing.image_dataset_from_directory(
    animal_data_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 1200 files belonging to 2 classes.
Using 960 files for training.
Found 1200 files belonging to 2 classes.
Using 240 files for validation.


In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

# ✅ Build the base model
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze pre-trained layers

# ✅ Add custom layers for classification
animal_model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(animal_train.class_names), activation='softmax')  # Output layer
])

# ✅ Compile the model
animal_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# ✅ Train the model
animal_model.fit(
    animal_train,
    validation_data=animal_val,
    epochs=5
)

Epoch 1/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.8701 - loss: 0.2220 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [ ]:
model_file_path = "/content/drive/MyDrive/ML_DataSet/saved_animal_model.keras"

In [ ]:
import os

# Define the directory path
model_dir = "/content/drive/MyDrive/ML_DataSet"

# Create the directory if it does not exist
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Now save the model
model_file_path = os.path.join(model_dir, "saved_animal_model.keras")
animal_model.save(model_file_path)
print(f"✅ Model saved to {model_file_path}")

✅ Model saved to /content/drive/MyDrive/ML_DataSet/saved_animal_model.keras


In [ ]:
import tensorflow as tf

# Load the saved .keras model
model = tf.keras.models.load_model('/content/drive/MyDrive/ML_DataSet/saved_animal_model.keras')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to a file
tflite_model_path = '/content/drive/MyDrive/ML_DataSet/saved_animal_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"✅ Model converted to TFLite format and saved to {tflite_model_path}")

Saved artifact at '/tmp/tmpglj2bfko'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_14')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  138163097432912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138163197083920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138163197086416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138163197098320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138163197086032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138163197086800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138163197090640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138163197092176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138163197091792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138163197085264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138163197090